# Comparison of Shrinkage and Selection Methods for Linear Regression

In this article we will look at seven popular methods for subset selection and shrinkage in linear regression. After an introduction to the topic justifying the need for such methods, we will look at each approach one by one, covering both mathematical properties and a Python application.

## Why shrink or subset and what does this mean?

In the linear regression context, subsetting means choosing a subset from available variables to include in the model, thus reducing its dimensionality. Shrinkage, on the other hand, means reducing the size of the coefficient estimates (shrinking them towards zero). Note that if a coefficient gets shrunk to exactly zero, the corresponding variable drops out of the model. Consequently, such a case can also be seen as a kind of subsetting.

Shrinkage and selection aim at improving upon the simple linear regression. There are two main reasons why it could need an improvement:

* **Prediction accuracy:** Linear regression estimates tend to have low bias and high variance. Reducing model complexity (the number of parameters that need to be estimated) results in reducing the variance at the cost of introducing more bias. If we could find the sweet spot where the total error, so the error resulting from bias plus the one from variance, is minizmized, we can improve the model's predictions.


* **Model's interpretability:** With too many predictors it is hard for a human to grasp all the relations between the variables. In some cases we would be willing to determing a small subset of variables with the strongest impact, thus sacrificing some details in order to get the big picture.

## Setup & Data Load

Before jumping straight to the methods themselves, let us first look at the data set we will be analysing. It comes from a study by Stamey et al. (1989) who investigated the impact of different clinical measurements on the level of prostate specific antigen (PSA). The task is to identify the risk factors for prostate cancer, based on a set if clinical and demographic variables. The data, together with some desciptions of the variables, can be found [on the website of Hastie's et al. "The elements of statistical learning" textbook](http://web.stanford.edu/~hastie/ElemStatLearn/), in the Data section.

We will start by importing the modules used throughout this article, loading the data and splitting it into training and testing sets, keeping the targets and the features separately. We will then discuss each of the shrinkage and selection methods, fit it to the training data and use the test set to check how well can it predict the PSA levels on new data.

In [1]:
# Import necessary modules and set options
import pandas as pd
import numpy as np
import itertools

from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV, ElasticNetCV, LarsCV
from sklearn.cross_decomposition import PLSRegression
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

import warnings
warnings.filterwarnings("ignore")

# Load data
data = pd.read_csv("prostate_data", sep = "\t")
print(data.head())

# Train-test split
y_train = np.array(data[data.train == "T"]['lpsa'])
y_test = np.array(data[data.train == "F"]['lpsa'])
X_train = np.array(data[data.train == "T"].drop(['lpsa', 'train'], axis=1))
X_test = np.array(data[data.train == "F"].drop(['lpsa', 'train'], axis=1))

     lcavol   lweight  age      lbph  svi       lcp  gleason  pgg45      lpsa  \
0 -0.579818  2.769459   50 -1.386294    0 -1.386294        6      0 -0.430783   
1 -0.994252  3.319626   58 -1.386294    0 -1.386294        6      0 -0.162519   
2 -0.510826  2.691243   74 -1.386294    0 -1.386294        7     20 -0.162519   
3 -1.203973  3.282789   58 -1.386294    0 -1.386294        6      0 -0.162519   
4  0.751416  3.432373   62 -1.386294    0 -1.386294        6      0  0.371564   

  train  
0     T  
1     T  
2     T  
3     T  
4     T  


## Linear Regression

Let us start with the simple linear regression, which will constitute our benchmark. It models the target variable, _y_, as a linear combination of _p_ predictors, or features _X_:

<p align="center">
<img src="img/linreg_model.png" width=200 style="display: block; margin: auto;" />
</p>

This model has _p_ + 2 parameters that have to be estimated from the training data:

* The _p_ feature $\beta$-coefficients, one per viariable, denoting their impacts on the target;
* One intercept parameter, denoted as $\beta_0$  above, which is the prediction in case all Xs are zero. It is not necessary to include it in the model, and indeed in some cases it should be dropped (e.g. if one wants to include a full set of dummies denoting levels of a categorical variable) but in general it gives the model more flexibility, as you will see in the next paragraph;
* One variance parameter of the Gaussian error term. 

These parameters are typically estimated using the Ordinary Least Square (OLS). OLS minimizes the sum of squared residuals, given by

<p align="center">
<img src="img/linreg_rss.png" width=300 style="display: block; margin: auto;" />
</p>

It is helpful to think about this minimization criterion graphically. With only one predictor _X_, we are in a 2D space, formed by this predictor and the target. In this setting, the model fits such a line in the _X-Y_ space that is the closest to all data points, with the proximity measured as the sum of squared vertical distances of all data points - see the left panel below. If there are two predictors, $X_1$ and $X_2$, the space grows to 3D and now the model fits a plane that is closest to all points in the 3D space - see the right panel below. With more than two features, the plane becomes the somewhat abstract hyperplane, but the idea is still the same.

<p align="center">
<img src="img/linreg_3d_pic.png" width=600 style="display: block; margin: auto;" />
</p>

The minimization problem described above turns out to have an analytical solution, and the $\beta$-parameters can be calculated as

<p align="center">
<img src="img/linreg_coefs.png" width=175 style="display: block; margin: auto;" />
</p>

Including a column of ones in the _X_ matrix allows to express the intercept part of the $\hat{\beta}$ vector in the formula above. The "hat" above the $\beta$ denotes that it is an estimated value, based on the training data.

In statistics, there are two critical characteristics of estimators to be considered: the bias and the variance. The bias is the difference between the true population parameter and the expected estimator. It measures the inaccuracy of the estimates. The variance, on the other hand, measures the spread between them.

<p align="center">
<img src="img/bias_vs_variance.jpg" width=400 style="display: block; margin: auto;" />
</p>

Clearly, both bias and variance can harm the model's predictive performance if they are too large. The linear regression, however, tends to suffer from variance, while having a low bias. This is especially the case if there are many predictive features in the model or if they are highly correlated with each other. **This is where subsetting and regularization come to rescue. They allow to reduce the variance at the cost of introducing some more bias, ulimately reducing the total error of the model.**

Before discussing these methods in detail, let us fit linear regression to out prostate data and check it's out-of-sample Mean Prediction Error (MAE).

In [2]:
linreg_model = LinearRegression(normalize=True).fit(X_train, y_train)
linreg_prediction = linreg_model.predict(X_test)
linreg_mae = np.mean(np.abs(y_test - linreg_prediction))
linreg_coefs = dict(
    zip(['Intercept'] + data.columns.tolist()[:-1], 
        np.round(np.concatenate((linreg_model.intercept_, linreg_model.coef_), axis=None), 3))
)

print('Linear Regression MAE: {}'.format(np.round(linreg_mae, 3)))
print('Linear Regression coefficients:')
linreg_coefs

Linear Regression MAE: 0.523
Linear Regression coefficients:


{'Intercept': 0.429,
 'lcavol': 0.577,
 'lweight': 0.614,
 'age': -0.019,
 'lbph': 0.145,
 'svi': 0.737,
 'lcp': -0.206,
 'gleason': -0.03,
 'pgg45': 0.009}

## Best Subset Regression

In [3]:
results = pd.DataFrame(columns=['num_features', 'features', 'MAE'])

# Loop over all possible numbers of features to be included
for num_vars in range(1, X_train.shape[1] + 1):
    # Loop over all possible subsets of size num_vars
    for subset in itertools.combinations(range(X_train.shape[1]), num_vars):
        subset = list(subset)
        linreg_model = LinearRegression(normalize=True).fit(X_train[:, subset], y_train)
        linreg_prediction = linreg_model.predict(X_test[:, subset])
        linreg_mae = np.mean(np.abs(y_test - linreg_prediction))
        results = results.append(pd.DataFrame([{'num_features': num_vars,
                                                'features': subset,
                                                'MAE': linreg_mae}]))

# Inspect best combinations
results = results.sort_values('MAE').reset_index()
print(results.head())

# Fit best model
best_subset_model = LinearRegression(normalize=True).fit(X_train[:, results['features'][0]], y_train)
best_subset_coefs = dict(
    zip(['Intercept'] + data.columns.tolist()[:-1], 
        np.round(np.concatenate((best_subset_model.intercept_, best_subset_model.coef_), axis=None), 3))
)

print('Best Subset Regression MAE: {}'.format(np.round(results['MAE'][0], 3)))
print('Best Subset Regression coefficients:')
best_subset_coefs

   index       MAE            features num_features
0      0  0.466876     [0, 1, 2, 4, 7]            5
1      0  0.467043  [0, 1, 2, 4, 6, 7]            6
2      0  0.471730     [0, 1, 2, 4, 6]            5
3      0  0.478344        [0, 1, 4, 7]            4
4      0  0.479609        [0, 1, 4, 6]            4
Best Subset Regression MAE: 0.467
Best Subset Regression coefficients:


{'Intercept': -0.599,
 'lcavol': 0.497,
 'lweight': 0.81,
 'age': -0.012,
 'lbph': 0.413,
 'svi': 0.005}

## Ridge Regression

In [4]:
ridge_cv = RidgeCV(normalize=True, alphas=np.logspace(-10, 1, 400))
ridge_model = ridge_cv.fit(X_train, y_train)
ridge_prediction = ridge_model.predict(X_test)
ridge_mae = np.mean(np.abs(y_test - ridge_prediction))
ridge_coefs = dict(
    zip(['Intercept'] + data.columns.tolist()[:-1], 
        np.round(np.concatenate((ridge_model.intercept_, ridge_model.coef_), axis=None), 3))
)

print('Ridge Regression MAE: {}'.format(np.round(ridge_mae, 3)))
print('Ridge Regression coefficients:')
ridge_coefs

Ridge Regression MAE: 0.517
Ridge Regression coefficients:


{'Intercept': 0.155,
 'lcavol': 0.51,
 'lweight': 0.605,
 'age': -0.016,
 'lbph': 0.14,
 'svi': 0.692,
 'lcp': -0.134,
 'gleason': 0.009,
 'pgg45': 0.008}

## LASSO

In [5]:
lasso_cv = LassoCV(normalize=True, alphas=np.logspace(-10, 1, 400))
lasso_model = lasso_cv.fit(X_train, y_train)
lasso_prediction = lasso_model.predict(X_test)
lasso_mae = np.mean(np.abs(y_test - lasso_prediction))
lasso_coefs = dict(
    zip(['Intercept'] + data.columns.tolist()[:-1], 
        np.round(np.concatenate((lasso_model.intercept_, lasso_model.coef_), axis=None), 3))
)

print('LASSO MAE: {}'.format(np.round(lasso_mae, 3)))
print('LASSO coefficients:')
lasso_coefs

LASSO MAE: 0.5
LASSO coefficients:


{'Intercept': 0.074,
 'lcavol': 0.459,
 'lweight': 0.456,
 'age': -0.0,
 'lbph': 0.05,
 'svi': 0.352,
 'lcp': 0.0,
 'gleason': 0.0,
 'pgg45': 0.002}

## Elastic Net

In [6]:
elastic_net_cv = ElasticNetCV(normalize=True, alphas=np.logspace(-10, 1, 400), l1_ratio=np.linspace(0, 1, 100))
elastic_net_model = elastic_net_cv.fit(X_train, y_train)
elastic_net_prediction = elastic_net_model.predict(X_test)
elastic_net_mae = np.mean(np.abs(y_test - elastic_net_prediction))
elastic_net_coefs = dict(
    zip(['Intercept'] + data.columns.tolist()[:-1], 
        np.round(np.concatenate((elastic_net_model.intercept_, elastic_net_model.coef_), axis=None), 3))
)

print('Elastic Net MAE: {}'.format(np.round(elastic_net_mae, 3)))
print('Elastic Net coefficients:')
elastic_net_coefs

Elastic Net MAE: 0.5
Elastic Net coefficients:


{'Intercept': 0.074,
 'lcavol': 0.459,
 'lweight': 0.456,
 'age': -0.0,
 'lbph': 0.05,
 'svi': 0.352,
 'lcp': 0.0,
 'gleason': 0.0,
 'pgg45': 0.002}

## Principal Components Regression

In [7]:
regression_model = LinearRegression(normalize=True)
pca_model = PCA()
pipe = Pipeline(steps=[('pca', pca_model), ('least_squares', regression_model)])
param_grid = {'pca__n_components': range(1, 9)}
search = GridSearchCV(pipe, param_grid)
pcareg_model = search.fit(X_train, y_train)
pcareg_prediction = pcareg_model.predict(X_test)
pcareg_mae = np.mean(np.abs(y_test - pcareg_prediction))
n_comp = list(pcareg_model.best_params_.values())[0]
pcareg_coefs = dict(
   zip(['Intercept'] + ['PCA_comp_' + str(x) for x in range(1, n_comp + 1)], 
       np.round(np.concatenate((pcareg_model.best_estimator_.steps[1][1].intercept_, 
                                pcareg_model.best_estimator_.steps[1][1].coef_), axis=None), 3))
)

print('Principal Components Regression MAE: {}'.format(np.round(pcareg_mae, 3)))
print('Principal Components Regression coefficients:')
pcareg_coefs

Principal Components Regression MAE: 0.551
Principal Components Regression coefficients:


{'Intercept': 2.452,
 'PCA_comp_1': 0.019,
 'PCA_comp_2': -0.018,
 'PCA_comp_3': -0.114,
 'PCA_comp_4': 0.495,
 'PCA_comp_5': 0.513,
 'PCA_comp_6': -0.46}

## Partial Least Squares

In [8]:
pls_model_setup = PLSRegression(scale=True)
param_grid = {'n_components': range(1, 9)}
search = GridSearchCV(pls_model_setup, param_grid)
pls_model = search.fit(X_train, y_train)
pls_prediction = pls_model.predict(X_test)
pls_mae = np.mean(np.abs(y_test - pls_prediction))
pls_coefs = dict(
  zip(data.columns.tolist()[:-1], 
      np.round(np.concatenate((pls_model.best_estimator_.coef_), axis=None), 3))
)

print('Partial Least Squares Regression MAE: {}'.format(np.round(pls_mae, 3)))
print('Partial Least Squares Regression coefficients:')
pls_coefs

Partial Least Squares Regression MAE: 1.008
Partial Least Squares Regression coefficients:


{'lcavol': 0.281,
 'lweight': 0.186,
 'age': 0.087,
 'lbph': 0.101,
 'svi': 0.213,
 'lcp': 0.187,
 'gleason': 0.131,
 'pgg45': 0.171}

## Least Angle Regression

In [9]:
LAR_cv = LarsCV(normalize=True)
LAR_model = LAR_cv.fit(X_train, y_train)
LAR_prediction = LAR_model.predict(X_test)
LAR_mae = np.mean(np.abs(y_test - LAR_prediction))
LAR_coefs = dict(
    zip(['Intercept'] + data.columns.tolist()[:-1], 
        np.round(np.concatenate((LAR_model.intercept_, LAR_model.coef_), axis=None), 3))
)

print('Least Angle Regression MAE: {}'.format(np.round(LAR_mae, 3)))
print('Least Angle Regression coefficients:')
LAR_coefs

Least Angle Regression MAE: 0.499
Least Angle Regression coefficients:


{'Intercept': 0.05,
 'lcavol': 0.46,
 'lweight': 0.46,
 'age': 0.0,
 'lbph': 0.054,
 'svi': 0.362,
 'lcp': 0.0,
 'gleason': 0.0,
 'pgg45': 0.002}

## Recap & Conclusion